In [1]:
%run init.ipynb

Using TensorFlow backend.
/home/fanyixing/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/fanyixing/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 2633.34it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2358.67it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 1804738.53it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3791.66it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:10<00:00, 1788.70it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 38738.35it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 3821.69it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [3]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f08134b4da0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f0813d037b8>,
 'vocab_size': 16674,
 'embedding_input_dim': 16674,
 'input_shapes': [(10,), (40,)]}

In [4]:
model = mz.models.ConvKNRM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['filters'] = 128 
model.params['conv_activation_func'] = 'tanh' 
model.params['max_ngram'] = 3
model.params['use_crossmatch'] = True 
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
#model.backend.summary()

In [5]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [6]:
pred_x, pred_y = test_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y))

In [7]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=5, num_neg=1, batch_size=20)
len(train_generator)

255

In [ ]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
255/255 [==============================] - 47s 184ms/step - loss: 0.3839
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6045607115348546 - normalized_discounted_cumulative_gain@5(0.0): 0.6551911357360845 - mean_average_precision(0.0): 0.6161963540752681
Epoch 2/30
255/255 [==============================] - 51s 201ms/step - loss: 0.0949
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5924670551810475 - normalized_discounted_cumulative_gain@5(0.0): 0.6458352102003769 - mean_average_precision(0.0): 0.6010633279040125
Epoch 3/30
255/255 [==============================] - 51s 198ms/step - loss: 0.0517
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6184166544728893 - normalized_discounted_cumulative_gain@5(0.0): 0.6666566279990483 - mean_average_precision(0.0): 0.6212755577262572
Epoch 4/30
255/255 [==============================] - 51s 199ms/step - loss: 0.0263
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6135025752199578 - nor